In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import datetime
import neurokit2 as nk
import numpy as np
%matplotlib inline

In [14]:
SAMPLING_RATE = 1000

In [15]:
df = pd.read_csv('../data/1222325cFnorm.csv')

In [16]:
ecg = df["ECG"].values.tolist()

### Downsample the data

In [17]:
ecg_ds = nk.signal_resample(ecg, sampling_rate=2000, desired_sampling_rate=SAMPLING_RATE)

In [18]:
ecg_df = pd.DataFrame(ecg_ds, columns=['ECG'])

In [19]:
ecg_df

,ECG
0,0.015600
1,0.015573
2,0.015637
3,0.015685
4,0.015700
...,...
35331833,-0.000582
35331834,-0.000804
35331835,-0.000689
35331836,-0.000695


In [22]:
# Take first n minutes
minutes = 15

n = minutes*60*SAMPLING_RATE
df = ecg_df[:n]

In [26]:
print(f"Signal duration: {datetime.timedelta(seconds=len(df.index)/SAMPLING_RATE)}")

Signal duration: 0:15:00


## Define LF/HF value ranges

### Paper: Herzig et al., Reproducibility of Heart Rate Variability Is Parameter and Sleep Stage Dependent (2017)

In [27]:
# stage_2 = {'min': 0.68, 'median': 1.11, 'max': 2.02}
# sws = {'min': 0.31 , 'median': 0.51, 'max': 0.90}
nrem = {'min': 0.31, 'median': (2.02-0.31)/2 , 'max': 2.02}
rem = {'min': 1.30, 'median': 2.02 , 'max': 3.22}

### Paper: Ako et al., Correlation between electroencephalography and heart rate variability during sleep (2003)

In [ ]:
stage_1 = {'min': 2.30-0.29, 'median': 2.30, 'max': 2.30+0.29}
stage_2 = {'min': 1.85-0.09 , 'median': 1.85 , 'max': 1.85+0.09}
stage_3 = {'min': 0.78-0.06 , 'median': 0.78, 'max': 0.78+0.06}
stage_4 = {'min': 0.86-0.14, 'median': 0.86, 'max': 0.86+0.14}
rem = {'min': 2.51-0.17, 'median': 2.51, 'max': 2.51+0.17}

In [28]:
start = 0
# Number of data points in 5 minutes
window_size = 5*60*SAMPLING_RATE
values = []

while window_size <= df.index[-1]+1:
    stages = []
    print(f"df index -1 + 1: {df.index[-1]+1}")

    df_5_min = df[start:window_size]
    ecg = df_5_min["ECG"].values.tolist()
    # Filter the data with ranges specified from Barbara
    filter_ecg = nk.signal_filter(ecg, sampling_rate=SAMPLING_RATE, lowcut=0.5, highcut=150)
    cleaned = nk.ecg_clean(filter_ecg, sampling_rate=SAMPLING_RATE, method="neurokit")
    peaks, info = nk.ecg_peaks(cleaned, sampling_rate=SAMPLING_RATE, method="neurokit")

    hrv_indices = nk.hrv(peaks, sampling_rate=SAMPLING_RATE)
    hrv_welch = nk.hrv_frequency(peaks, sampling_rate=SAMPLING_RATE, psd_method="welch")

    if  nrem['min'] <= hrv_indices['HRV_LFHF'][0] <= nrem['max'] + 1:
        stages.append('nrem')

    elif rem['min'] <= hrv_indices['HRV_LFHF'][0] <= rem['max'] + 1:
        stages.append('rem')

    

    values.append({
        'start_id': start,
        'end_id': window_size,
        'LF/HF': hrv_welch['HRV_LFHF'][0],
        'SD': hrv_indices['HRV_SDNN'][0],
        'stages': stages,
        'color': None
    })

    

    start = window_size
    window_size += window_size

    print(f"start: {start}")
    print(f"window size: {window_size}")


    


df index -1: 899999
start: 0
window size: 300000
300000
df index -1: 899999
start: 300000
window size: 600000
600000


In [29]:
values

[{'start_id': 0,
  'end_id': 300000,
  'LF/HF': 1.2104719065647447,
  'SD': 464.49439264217165,
  'stages': ['nrem'],
  'color': None},
 {'start_id': 300000,
  'end_id': 600000,
  'LF/HF': 0.5692369237953601,
  'SD': 142.4711915588191,
  'stages': ['nrem'],
  'color': None}]